# 2018

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime
import pickle

from loadandconvert import loadCSIfluxperiod, deltatogas,f_gg_water, f_gg_ice,alpha18O_Ell,alphaD_Ell,nearest,alpha18O_Maj,alphaD_MN

#for plotting
import mpl_axes_aligner.align as align
from loadandconvert import fontsize,plot1,plot2
fontsize()

In [8]:
# import the data

# the PROMICE dataset in 10 min steps --> only needed for emissivity correction
pathgeus10min='/Users/swa048/forServer/Meteo/PROMICE_GEUS/2019/EGP_2020_06_24_v03/EGP_inst_v03.txt'
mydateparser_min = lambda x: datetime.datetime.strptime(x, "%Y %m %d %H %M")
dfgeus=pd.read_csv(pathgeus10min,delim_whitespace=True,index_col=(0),parse_dates=[[0,1,2,3,4]],infer_datetime_format=True ,date_parser=mydateparser_min,na_values=['NAN',-999.00],usecols=(0,1,2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,21,22) )
dfgeus.rename(inplace=True,columns={ 
        'AirPressure(hPa)': 'P',
        'AirTemperature(C)': 'Ta',
        'RelativeHumidity(%)':'RH', #is this already calculated with respect to ice?
        'WindSpeed(m/s)':'WS',
        'SpecificHumidity(g/kg)':'q',
        'WindDirection(d)':'WD',
        'ShortwaveRadiationDown(W/m2)':'SWdown',
        'ShortwaveRadiationDown_Cor(W/m2)':'SWdowncorr',
        'ShortwaveRadiationUp(W/m2)':'SWup',
        'ShortwaveRadiationUp_Cor(W/m2)':'SWupcorr',
        'LongwaveRadiationDown(W/m2)':'LWdown',
        'LongwaveRadiationUp(W/m2)':'LWup',
        'SurfaceTemperature(C)':'Ts'})
dfgeus=dfgeus[~dfgeus.index.duplicated(keep='first')]
dfgeus=dfgeus.loc['2018',:]
dfgeus.DayOfYear=dfgeus.DayOfYear-1  #year starting with DAY 0, not with DAY 1
dfgeus.loc[:,'RHs']=dfgeus.RH*(f_gg_ice(dfgeus.Ta)/f_gg_ice(dfgeus.Ts))/100

# flux data frpom 2018
dfflux=pd.read_csv('/Users/swa048/forServer/Meteo/EC/2018/EGRIP/onlinefluxes10min/CR3000_ec_scfd.dat',index_col=0,usecols=([0,1,2,3,4,5,17,24,25,26,27,28,30]),skiprows=[0,2,3],parse_dates=True,na_values=['NAN',-7999])
dfflux.rename(inplace=True,columns={ 
        'horiz_wind_spd' : 'WS_hori', #EC100 temp probe
    'result_wind_spd':'WS', #EC100 temp probe
    'wind_dir' : 'WD', #T_sonic
    'RH_2_1_1':'RH_w2', #T_sonic
    'Ts_Avg':'Tsonic',
    })

# 2m data vapor
df2m=pd.read_csv('/Users/swa048/forServer/Vapour/2018/processed_data_combined/EG18_level180cm.txt',index_col=0,parse_dates=True,na_values=['NAN'])

dfST_05=pd.read_csv('/Users/swa048/forServer/Snow/Isotopes/2018/data_measuredsamples/ST/ST_05cm.txt',index_col=0,parse_dates=True,na_values=['NAN'],delim_whitespace=True)
dfST_1=pd.read_csv('/Users/swa048/forServer/Snow/Isotopes/2018/data_measuredsamples/ST/ST_1cm.txt',index_col=0,parse_dates=True,na_values=['NAN'],delim_whitespace=True)
dfST_2=pd.read_csv('/Users/swa048/forServer/Snow/Isotopes/2018/data_measuredsamples/ST/ST_2cm.txt',index_col=0,parse_dates=True,na_values=['NAN'],delim_whitespace=True)



In [9]:
dfflux

,RECORD,LE,Hs,tau,u_star,Uz_Ts,ln_vh_Avg,Tsonic,WS_hori,WS,wd,wnd_dir_compass
TIMESTAMP,,,,,,,,,,,,
2016-06-14 12:10:00,0,NaN,4.348480,0.020960,0.144178,0.004291,NaN,-7.526406,3.177135,3.163882,348.00970,168.0098
2016-06-14 12:20:00,1,8.742919,4.033833,0.020036,0.140966,0.003981,6.575305,-7.526401,3.349077,3.337338,348.11880,168.1188
2016-06-14 12:30:00,2,6.823554,4.872240,0.014515,0.119983,0.004808,6.578306,-7.520228,3.393901,3.377440,343.48480,163.4848
2016-06-14 12:40:00,3,7.323289,6.037766,0.018601,0.135825,0.005958,6.569839,-7.410271,3.237962,3.228435,340.84660,160.8466
2016-06-14 12:50:00,4,7.178411,5.090959,0.020632,0.143046,0.005024,6.561572,-7.214272,3.358988,3.344656,339.42820,159.4282
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-08-04 11:10:00,7322,4.193546,2.264487,0.041204,0.202153,0.002235,6.699192,-8.300155,5.030932,5.013360,46.09123,226.0912
2016-08-04 11:20:00,7323,4.271956,2.388350,0.037313,0.192370,0.002357,6.700994,-8.215418,5.055803,5.040335,45.53899,225.5390
2016-08-04 11:30:00,7324,4.273283,1.509579,0.043437,0.207558,0.001490,6.703388,-8.221793,4.979212,4.964911,46.04855,226.0486
